# To extract tables from arxiv PDFs
Started: Oct 9th- Last edit: Oct 10th
By Shooby

In [1]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR
from img2table.ocr import EasyOCR
import pandas as pd
import os
easyocr = EasyOCR(lang=["en"])


In [2]:
def remove_enter_row(row):
    ''' This module is a preprocessing to further split the table 
    if there are \n left in the data'''
    
    try:
        numenters = row.values[0].count('\n')
        for i in range(numenters):
            new_row = row.copy()
            for col, value in row.items():
                split_values = value.split('\n')
                new_row[col] = split_values[1]
                row[col]=split_values[0]
        return[row,new_row]
    except:
        return [row]
        
def remove_character(string, character):
  if character not in string:
    return string
  return string.replace(character, "")
    
def cleansave(extracted_tables,dir):
    for page, tables in extracted_tables.items():
        for idx, table in enumerate(tables):
            print('-----------------------------------------------------------------------')
            print('Title:'+str(table.title))
            if table.title:
                t = table.title
                if ' ' in t:
                    t = t.replace(' ','_')
                t = remove_character(t,'.')
                t = remove_character(t,"\\")
                t = remove_character(t,"/")
                t = remove_character(t,",")
                tablename = t+'.csv'
            else:
                tablename = 'table'+str(idx)+'_page'+str(page)+'.csv'
            tb = table.df
            processed_rows = [remove_enter_row(row) for index, row in tb.iterrows()]
            flattened_rows = [item for sublist in processed_rows for item in sublist]
            result_df = pd.DataFrame(flattened_rows, columns=tb.columns)
            dfp = result_df.fillna('-')
            dfp.to_csv(dir+'/'+tablename, index=False)
            print('-----------------------------------------------------------------------')


def pdf2csv(pdffile):
    pdf = PDF(src=pdffile)
    ocr = TesseractOCR(lang="eng")
    extracted_tables = pdf.extract_tables(ocr=ocr,
                                          implicit_rows=True,
                                          borderless_tables=False,
                                          min_confidence=80)
    dirname = pdffile.replace('.pdf','_tables')
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
    cleansave(extracted_tables,dirname)

In [3]:
pdf2csv('2005.12507.pdf')

tesseract 5.3.3
 leptonica-1.83.1
  libgif 5.2.1 : libjpeg 8d (libjpeg-turbo 3.0.0) : libpng 1.6.40 : libtiff 4.6.0 : zlib 1.2.11 : libwebp 1.3.2 : libopenjp2 2.5.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found libarchive 3.7.2 zlib/1.2.11 liblzma/5.4.4 bz2lib/1.0.8 liblz4/1.9.4 libzstd/1.5.5
 Found libcurl/7.87.0 SecureTransport (LibreSSL/3.3.6) zlib/1.2.11 nghttp2/1.45.1
-----------------------------------------------------------------------
Title:5
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Title:Table 4
priors used for diﬀerent free physical parameters and the ﬁxed parameters used in the ﬁt.
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Title:1.0
-----------------------------------------------------------------------
-------------------------------------------------------------

OSError: [Errno 63] File name too long: '2005.12507_tables/Table_7\n+_Table_of_the_candidate_galaxies_and_their_likelihood_measures_from_diﬀerent_methods_The_and_next_to_the_IDs_show_the_candidates_∗\nfound_in_the_primary_and_secondary_samples_reported_in_Merlin_et_al_(2019)_respectively_The_next_to_the_redshift_indicates_those_with_∗\navailable_spectroscopic_redshifts_The_BBG_UVJ_and_SED_stand_for_likelihood_based_on_the_observed_color_UVJ_and_SED_ﬁtting\nrespectively_MIPS_and_X-ray_columns_correspond_to_the_detection_in_the_MIPS_24_µm_and_X-ray_is_log_and_Age_is_the_M∗_(MsM⊙)\nage_of_the_candidate_galaxy_in_Gyrs_The_galaxies_that_show_a_detection_in_MIPS_24_µm_and_X-ray_are_excluded_from_the\nnumberstellar_mass_densities_measurements_(ie_Figure_9).csv'

In [ ]:
pdf = PDF(src='20')
ocr = TesseractOCR(lang="eng")
extracted_tables = pdf.extract_tables(ocr=easyocr,implicit_rows=True,borderless_tables=False,min_confidence=30)
for page, tables in extracted_tables.items():
    for idx, table in enumerate(tables):
        print(table)
